In [323]:
# Importar librería para la conexión con MySQL
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer


pd.set_option('display.max_columns', None)

In [324]:
df = pd.read_csv('../../files/df_to_SQL.csv' , index_col = 0)

In [325]:
def insercion_datos (df):
    
    # Crea dos columnas extra: RecordDate y OverallSatisfaction
    df['OverallSatisfaction'] = df.apply(lambda x: int(round((x['EnvironmentSatisfaction'] + x['JobSatisfaction'] + x['RelationshipSatisfaction']) / 3)), axis=1)

    # Pone todos los datos en .lower()
    df = df.apply(lambda col : col.map(lambda x : x.lower() if isinstance(x , str) else x))

    # Crea el DF con los datos de la tabla Employee
    df_employee = df.loc[: , ['EmployeeID' , 'Age' , 'DateBirth' , 'Gender' , 'MaritalStatus', 'NumberChildren']]
    df_employee.drop_duplicates(inplace=True)
    df_employee = df_employee.reset_index()
    df_employee.drop('index' , axis = 1 , inplace = True)

    # Crea el DF con los datos de la tabla EmployeeDetails
    df_employeedetails = df.loc[: , ['EmployeeID' , 'Education' , 'EducationField' , 'NumCompaniesWorked', 'TotalWorkingYears']]
    df_employeedetails.drop_duplicates(inplace=True)
    df_employeedetails = df_employeedetails.reset_index()
    df_employeedetails.drop('index' , axis = 1 , inplace = True)

    # Crea el DF con los datos de la tabla Surveys
    df_surveys = df.loc[: , ['EmployeeID' , 'EnvironmentSatisfaction' , 'JobSatisfaction' , 'RelationshipSatisfaction' , 'OverallSatisfaction', 'WorkLifeBalance']]

    # Crea el DF con los datos de la tabla SupervisorSurveys
    df_supsurveys = df.loc[: , ['EmployeeID' , 'JobInvolvement' , 'PerformanceRating']]

    # Crea el DF con los datos de la tabla EmploymentGeneral
    df_employgen = df.loc[: , ['EmployeeID' , 'Attrition' , 'OverTime' , 'DistanceFromHome', 'RemoteWork', 'BusinessTravel' , 'TrainingTimesLastYear' , 'StandardHours']]
    df_employgen['OverTime'] = df_employgen['OverTime'].apply(lambda x : 'Yes' if x == float(1) else ('No' if x == float(0) else np.nan))
    df_employgen['RemoteWork'] = df_employgen['RemoteWork'].apply(lambda x : 'Yes' if x == 1 else ('No' if x == 0 else np.nan))
    df_employgen['BusinessTravel']= df_employgen['BusinessTravel'].apply(lambda x : x.replace('_' , ' ').replace('-', ' ') if isinstance(x , str) else x)

    # Crea el DF con los datos de la tabla EmploymentTeams
    df_employteams = df.loc[: , ['EmployeeID' , 'Department' , 'JobRole' , 'JobLevel']]

    # Crea el DF con los datos de la tabla EmploymentExperience
    df_employexp = df.loc[: , ['EmployeeID' , 'YearsAtCompany' , 'YearsSinceLastPromotion' , 'YearsInCurrentRole' , 'YearsWithCurrManager']]

    # Crea el DF con los datos de la tabla SalaryDetails
    df_salary = df.loc[: , ['EmployeeID' , 'HourlyRate' , 'DailyRate' , 'MonthlyRate' , 'MonthlyIncome' , 'PercentSalaryHike' , 'StockOptionLevel']]
    
    # Crea la conexión con la base de datos y los inserta
    try:
        cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1' , database='abc_corporation')
        
        mycursor = cnx.cursor()
        
       # Definir los DataFrames y los nombres de las tablas
        df_list = [df_employee, df_employeedetails, df_surveys, df_supsurveys, df_employgen, df_employteams, df_employexp, df_salary]
        table_names = ['employee', 'employeedetails', 'surveys', 'supervisorsurveys', 'employmentgeneral', 'employmentteams', 'employmentexperience', 'salarydetails']

        for dataframe, table in zip(df_list, table_names):
            # Construir la lista de columnas y valores correspondientes
            columns = ', '.join(dataframe.columns)
            values_list = []
            
            # Iterar por cada fila del DataFrame
            for index, row in dataframe.iterrows():
                values = tuple(row[col] for col in dataframe.columns)
                values_list.append(values)

            try: 
                # Construir la consulta SQL de inserción
                insert_query = f"""
                    INSERT INTO {table} ({columns})
                    VALUES ({', '.join(['%s'] * len(dataframe.columns))})
                """

                # Ejecutar la consulta SQL de inserción para cada fila en values_list
                mycursor.executemany(insert_query, values_list)
            
            except mysql.connector.Error as err:
                # Imprimir el error junto con la fila actual y la tabla
                print(f"Error al insertar en la tabla {table} en la fila {index}: {err}")
        
        cnx.commit()
        
    except mysql.connector.Error as err:

        # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        
        # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        
        # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
        else:
            print(err)    

In [326]:
insercion_datos(df)